# Semantic Kernel na may OpenBnB MCP Server Integration

Ipinapakita ng notebook na ito kung paano gamitin ang Semantic Kernel kasama ang aktwal na OpenBnB MCP server upang maghanap ng mga totoong Airbnb accommodations gamit ang MCPStdioPlugin. Para sa LLM Access, gumagamit ito ng Azure AI Foundry. Upang i-setup ang iyong mga environment variable, maaari mong sundan ang [Setup Lesson](/00-course-setup/README.md)


In [ ]:
# Import cell - Updated imports
import json
import os
import asyncio

from dotenv import load_dotenv
from IPython.display import display, HTML
from typing import Annotated

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.connectors.mcp import MCPStdioPlugin
from semantic_kernel.contents import FunctionCallContent, FunctionResultContent, StreamingTextContent

## Paglikha ng MCP Plugin Connection

Ikokonekta natin ang [OpenBnB MCP server](https://github.com/openbnb-org/mcp-server-airbnb) gamit ang MCPStdioPlugin. Ang server na ito ay nagbibigay ng functionality para sa paghahanap ng Airbnb sa pamamagitan ng @openbnb/mcp-server-airbnb package.


## Paglikha ng Kliyente

Sa halimbawang ito, gagamitin natin ang Azure AI Foundry para sa ating LLM access. Siguraduhing maayos ang pagkaka-set up ng iyong mga environment variable.


## Konfigurasyon ng Kapaligiran

I-configure ang mga setting ng Azure OpenAI. Siguraduhing naitakda mo ang mga sumusunod na environment variables:
- `AZURE_OPENAI_CHAT_DEPLOYMENT_NAME`
- `AZURE_OPENAI_ENDPOINT`
- `AZURE_OPENAI_API_KEY`


In [ ]:
# Creating the Client cell - Updated for Azure
load_dotenv()

# Azure OpenAI configuration
# Ensure these environment variables are set:
# - AZURE_OPENAI_CHAT_DEPLOYMENT_NAME
# - AZURE_OPENAI_ENDPOINT
# - AZURE_OPENAI_API_KEY (optional if using DefaultAzureCredential)

chat_completion_service = AzureChatCompletion(
    deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
    endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    # Optional - will use DefaultAzureCredential if not set
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
)

## Pag-unawa sa OpenBnB MCP Integration

Ang notebook na ito ay kumokonekta sa **tunay na OpenBnB MCP server** na nagbibigay ng aktwal na functionality sa paghahanap ng Airbnb.

### Paano ito gumagana:

1. **MCPStdioPlugin**: Gumagamit ng standard input/output na komunikasyon sa MCP server
2. **Tunay na NPM Package**: Dina-download at pinapatakbo ang `@openbnb/mcp-server-airbnb` gamit ang npx
3. **Live Data**: Nagbabalik ng aktwal na data ng mga property sa Airbnb mula sa kanilang APIs
4. **Function Discovery**: Awtomatikong natutuklasan ng agent ang mga available na function mula sa MCP server

### Mga Available na Function:

Karaniwang ibinibigay ng OpenBnB MCP server ang:
- **search_listings** - Maghanap ng mga property sa Airbnb batay sa lokasyon at pamantayan
- **get_listing_details** - Kumuha ng detalyadong impormasyon tungkol sa mga partikular na property
- **check_availability** - Suriin ang availability para sa mga partikular na petsa
- **get_reviews** - Kunin ang mga review para sa mga property
- **get_host_info** - Kumuha ng impormasyon tungkol sa mga host ng property

### Mga Kinakailangan:

- **Node.js** na naka-install sa iyong sistema
- **Internet connection** para ma-download ang MCP server package
- **NPX** na available (kasama sa Node.js)

### Pagsubok sa Koneksyon:

Maaari mong subukan ang MCP server nang manu-mano sa pamamagitan ng pagpapatakbo ng:
```bash
npx -y @openbnb/mcp-server-airbnb
```

Ito ay magda-download at magsisimula ng OpenBnB MCP server, na kung saan ay kokonekta ang Semantic Kernel para sa aktwal na data ng Airbnb.


## Pagpapatakbo ng Agent gamit ang OpenBnB MCP Server

Ngayon, patatakbuhin natin ang AI Agent na kumokonekta sa OpenBnB MCP server upang maghanap ng totoong Airbnb na tirahan sa Stockholm para sa 2 matatanda at 1 bata. Malaya kang baguhin ang `user_inputs` list upang i-adjust ang mga pamantayan sa paghahanap.


In [ ]:
# Main execution cell - Enhanced with proper HTML rendering and MCP tool logging
# User requests for Airbnb search
user_inputs = [
    "Find Airbnb in Stockholm for 2 adults 1 kid",
]


async def main():
    """Main function to run the MCP-enabled agent with real OpenBnB server using Azure OpenAI"""

    try:
        # Create MCP plugin connection to real OpenBnB server
        async with MCPStdioPlugin(
            name="AirbnbSearch",
            description="Search for Airbnb accommodations using OpenBnB MCP server",
            command="npx",
            args=["-y", "@openbnb/mcp-server-airbnb", "--ignore-robots-txt"],
        ) as airbnb_plugin:

            print("🔧 MCP Plugin created and connected")

            # Load tools for function discovery
            await airbnb_plugin.load_tools()
            await asyncio.sleep(3)  # Give more time for initialization
            print("✅ Tools loaded from MCP server")

            # Debug: Check what tools were loaded
            if hasattr(airbnb_plugin, '_tools'):
                print(f"📋 Internal tools: {airbnb_plugin._tools}")

            # Verify available functions
            funcs = [attr for attr in dir(airbnb_plugin)
                     if callable(getattr(airbnb_plugin, attr))
                     and attr in ['airbnb_search', 'airbnb_listing_details']]
            print(f"📋 Available functions: {funcs}")

            # Create agent with Azure OpenAI service
            agent = ChatCompletionAgent(
                service=AzureChatCompletion(),  # Use default constructor
                name="AirbnbAgent",
                instructions="""You are an Airbnb search assistant. Use the airbnb_search function to find properties. 
                Format results in a clear HTML table with columns for property name, price, rating, and link.""",
                plugins=[airbnb_plugin],
            )

            print("🤖 Agent created with Azure OpenAI")

            # Process each user input
            thread: ChatHistoryAgentThread | None = None

            for user_input in user_inputs:
                print(f"\n🔍 Processing request: {user_input}")
                
                # Track MCP tool usage
                mcp_tools_used = []
                function_calls_log = []
                
                # Try streaming to capture function calls
                try:
                    agent_name = None
                    full_response = []
                    current_function_name = None
                    argument_buffer = ""
                    
                    async for response in agent.invoke_stream(
                        messages=user_input,
                        thread=thread,
                    ):
                        thread = response.thread
                        agent_name = response.name
                        
                        for item in response.items:
                            # Log function calls
                            if isinstance(item, FunctionCallContent):
                                if item.function_name:
                                    current_function_name = item.function_name
                                    mcp_tools_used.append(item.function_name)
                                    print(f"\n🔧 MCP Tool Selected: {item.function_name}")
                                    
                                if isinstance(item.arguments, str):
                                    argument_buffer += item.arguments
                            
                            # Log function results
                            elif isinstance(item, FunctionResultContent):
                                if current_function_name:
                                    try:
                                        args = json.loads(argument_buffer.strip()) if argument_buffer else {}
                                    except:
                                        args = {"raw": argument_buffer}
                                    
                                    function_calls_log.append({
                                        "function": current_function_name,
                                        "arguments": args,
                                        "timestamp": asyncio.get_event_loop().time()
                                    })
                                    
                                    print(f"   📍 Arguments: {json.dumps(args, indent=2)}")
                                    print(f"   ✅ MCP Tool Executed Successfully")
                                    
                                    current_function_name = None
                                    argument_buffer = ""
                            
                            # Collect response text
                            elif isinstance(item, StreamingTextContent) and item.text:
                                full_response.append(item.text)
                    
                    # Join the full response
                    response_text = ''.join(full_response)
                    
                except Exception as e:
                    print(f"⚠️ Streaming failed, using get_response: {str(e)[:100]}")
                    # Fallback to non-streaming
                    response = await agent.get_response(messages=user_input, thread=thread)
                    thread = response.thread
                    response_text = str(response)
                    agent_name = response.name
                
                
                # Process the response to ensure HTML tables render correctly
                # Remove any markdown code blocks around HTML
                response_text = response_text.replace('```html', '').replace('```', '')
                
                # Ensure proper HTML structure for tables
                if '<table' in response_text.lower():
                    # Add CSS styling for better table rendering
                    table_css = """
                    <style>
                        .airbnb-results table {
                            border-collapse: collapse;
                            width: 100%;
                            margin: 10px 0;
                        }
                        .airbnb-results th, .airbnb-results td {
                            border: 1px solid #ddd;
                            padding: 8px;
                            text-align: left;
                        }
                        .airbnb-results th {
                            background-color: #f2f2f2;
                            font-weight: bold;
                        }
                        .airbnb-results tr:nth-child(even) {
                            background-color: #f9f9f9;
                        }
                        .airbnb-results a {
                            color: #1976d2;
                            text-decoration: none;
                        }
                        .airbnb-results a:hover {
                            text-decoration: underline;
                        }
                    </style>
                    """
                    response_text = f'{table_css}<div class="airbnb-results">{response_text}</div>'
                
                # Build the complete HTML output
                html_output = f"""
                <div style='margin:10px; padding:10px; border-left:3px solid #2E8B57; background:#F0F8FF;'>
                    <strong>User:</strong> {user_input}
                </div>
                """
                
                # Add function call details if available
                if function_calls_log:
                    details_html = "<details style='margin:10px; padding:10px; background:#f5f5f5;'>"
                    details_html += "<summary><strong>📊 Function Call Details</strong></summary>"
                    details_html += "<pre style='background:#fff; padding:10px; overflow-x:auto;'>"
                    for call in function_calls_log:
                        details_html += f"Function: {call['function']}\n"
                        details_html += f"Arguments: {json.dumps(call['arguments'], indent=2)}\n"
                        details_html += "---\n"
                    details_html += "</pre></details>"
                    html_output += details_html
                
                # Add the agent's response with proper HTML rendering
                html_output += f"""
                <div style='margin:10px; padding:15px; border-left:3px solid #1E90FF; background:#FFFFFF;'>
                    <strong>{agent_name}:</strong><br>
                    {response_text}
                </div>
                """
                
                # Display the HTML with proper rendering
                display(HTML(html_output))
                
                
    except Exception as e:
        print(f"❌ Error: {str(e)}")
        import traceback
        traceback.print_exc()

print("🚀 Starting with Azure OpenAI...")
await main()
print("✅ Done!")

Buod  
Binabati kita! Matagumpay mong nabuo ang isang AI agent na kayang mag-integrate sa aktwal na paghahanap ng akomodasyon gamit ang Model Context Protocol (MCP):

Mga Teknolohiyang Ginamit:  
- Semantic Kernel - Para sa paggawa ng matatalinong agent gamit ang Azure OpenAI  
- Azure AI Foundry - Para sa kakayahan ng LLM at chat completion  
- MCP (Model Context Protocol) - Para sa standardisadong integrasyon ng mga tool  
- OpenBnB MCP Server - Para sa aktwal na functionality ng Airbnb search  
- Node.js/NPX - Para sa pagpapatakbo ng external MCP server  

Ano ang Iyong Natutunan:  
- MCP Integration: Pagkonekta ng Semantic Kernel agents sa mga external MCP server  
- Real-time Data Access: Paghahanap ng aktwal na mga ari-arian sa Airbnb gamit ang live APIs  
- Protocol Communication: Paggamit ng stdio communication sa pagitan ng agent at MCP server  
- Function Discovery: Awtomatikong pagtuklas ng mga available na function mula sa MCP servers  
- Streaming Responses: Pagkuha at pag-log ng mga function call sa real-time  
- HTML Rendering: Pag-format ng mga sagot ng agent gamit ang mga styled table at interactive na display  

Mga Susunod na Hakbang:  
- Mag-integrate ng karagdagang MCP servers (panahon, flights, mga restawran)  
- Bumuo ng multi-agent system na pinagsasama ang MCP at A2A protocols  
- Gumawa ng custom MCP servers para sa sarili mong mga data source  
- Magpatupad ng persistent conversation memory sa bawat session  
- I-deploy ang agent sa Azure Functions na may orchestration ng MCP server  
- Magdagdag ng user authentication at kakayahan sa booking  

Pangunahing Mga Bentahe ng MCP Architecture:  
- Standardization: Isang unibersal na protocol para sa pagkonekta ng AI agents sa mga external na tool  
- Real-time Data: Access sa live at napapanahong impormasyon mula sa iba't ibang serbisyo  
- Extensibility: Madaling integrasyon ng mga bagong data source at tool  
- Interoperability: Gumagana sa iba't ibang AI frameworks at agent platforms  
- Separation of Concerns: Malinaw na pagkakaiba sa pagitan ng AI logic at external na access sa data  



---

**Paunawa**:  
Ang dokumentong ito ay isinalin gamit ang AI translation service na [Co-op Translator](https://github.com/Azure/co-op-translator). Bagama't sinisikap naming maging tumpak, pakitandaan na ang mga awtomatikong pagsasalin ay maaaring maglaman ng mga pagkakamali o hindi pagkakatugma. Ang orihinal na dokumento sa kanyang katutubong wika ang dapat ituring na opisyal na sanggunian. Para sa mahalagang impormasyon, inirerekomenda ang propesyonal na pagsasalin ng tao. Hindi kami mananagot sa anumang hindi pagkakaunawaan o maling interpretasyon na dulot ng paggamit ng pagsasaling ito.
